# Running a CAT based on a synthetic correct/incorrect MIRT model

This example shows how to run a CAT based on a synthetic correct/incorrect
MIRT model.

Import order is important. We put ComputerAdaptiveTesting last so we get the
extra dependencies.

In [1]:
using Makie
import Pkg
import Random
using Distributions: Normal, cdf
using AlgebraOfGraphics
using ComputerAdaptiveTesting
using ComputerAdaptiveTesting.ExtraDistributions: NormalScaledLogistic
using ComputerAdaptiveTesting.Sim: auto_responder
using ComputerAdaptiveTesting.NextItemRules: DRuleItemCriterion
using ComputerAdaptiveTesting.TerminationConditions: FixedItemsTerminationCondition
using ComputerAdaptiveTesting.Aggregators: PriorAbilityEstimator, MeanAbilityEstimator, LikelihoodAbilityEstimator
using ComputerAdaptiveTesting.ItemBanks
using ComputerAdaptiveTesting.Integrators
import ComputerAdaptiveTesting.IntegralCoeffs
using CATPlots

@automakie()

Now we are read to generate our synthetic data using the supplied DummyData
module. We generate an item bank with 100 items and fake responses for 3
testees.

In [2]:
dims = 2
using ComputerAdaptiveTesting.DummyData: dummy_full, std_mv_normal, SimpleItemBankSpec, StdModel4PL
using ComputerAdaptiveTesting.MathTraits
using ComputerAdaptiveTesting.Responses: BooleanResponse

TODO: pass in dims

In [3]:
(item_bank, question_labels, abilities, responses) = dummy_full(
    Random.default_rng(42),
    SimpleItemBankSpec(StdModel4PL(), VectorContinuousDomain(), BooleanResponse()),
    dims;
    num_questions=10,
    num_testees=2
)

(ComputerAdaptiveTesting.ItemBanks.SlipItemBank{ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}}([0.0, 0.0, 0.16470051803438313, 0.20434519032982787, 0.2232106250114871, 0.0, 0.0, 0.368148798025053, 0.0, 0.0], ComputerAdaptiveTesting.ItemBanks.GuessItemBank{ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}}([0.0, 0.0, 0.1360736650138485, 0.40680966694187276, 0.03208125905881864, 0.07530994299752368, 0.006529406000667266, 0.12318232624245434, 0.0, 0.1779011051506347], ComputerAdaptiveTesting.ItemBanks.CdfMirtItemBank{Distributions.Logistic{Float64}}(Distributions.Logistic{Float64}(μ=0.0, θ=0.5875440658049353), [0.17586707655648473, 0.5281914214039702, -1.743425607159016, -0.2925134548881956, 1.03171287250986, 1.289362557146196, 0.9991446736933141, -0.5373256559130092, 0.037023537533112154, 0.11792593587253174], [1.039860452290571 0.9585093056236711 … 0.7495938488114743 0.

Simulate a CAT for each testee and record it using CatRecorder.
CatRecorder collects information which can be used to draw different types of plots.

In [4]:
max_questions = 9
integrator = CubaIntegrator([-6.0, -6.0], [6.0, 6.0], CubaVegas(); rtol=1e-2)
ability_estimator = MeanAbilityEstimator(PriorAbilityEstimator(std_mv_normal(dims)), integrator)
rules = CatRules(
    ability_estimator,
    DRuleItemCriterion(ability_estimator),
    FixedItemsTerminationCondition(max_questions)
)

CatRules
  next_item: ComputerAdaptiveTesting.NextItemRules.ItemStrategyNextItemRule{ComputerAdaptiveTesting.NextItemRules.ExhaustiveSearch1Ply, ComputerAdaptiveTesting.NextItemRules.DRuleItemCriterion{ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integrators.CubaIntegrator{ComputerAdaptiveTesting.Integrators.CubaVegas, Base.Pairs{Symbol, Float64, Tuple{Symbol}, NamedTuple{(:rtol,), Tuple{Float64}}}}}}}}
  termination_condition: ComputerAdaptiveTesting.TerminationConditions.FixedItemsTerminationCondition
  ability_estimator: ComputerAdaptiveTesting.Aggregators.MeanAbilityEstimator{ComputerAdaptiveTesting.Aggregators.PriorAbilityEstimator{Distributions.ZeroMeanIsoNormal{Tuple{Base.OneTo{Int64}}}}, ComputerAdaptiveTesting.Aggregators.FunctionIntegrator{ComputerAdaptiveTesting.Integra

XXX: We shouldn't need to specify xs here since the distributions are not used -- rework

In [5]:
points = 3
xs = repeat(range(-2.5, 2.5, length=points)', dims, 1)
raw_estimator = LikelihoodAbilityEstimator()
recorder = CatRecorder(xs, responses, integrator, raw_estimator, ability_estimator, abilities)
for testee_idx in axes(responses, 2)
    @debug "Running for testee" testee_idx
    tracked_responses, θ = run_cat(
        CatLoopConfig(
            rules=rules,
            get_response=auto_responder(@view responses[:, testee_idx]),
            new_response_callback=(tracked_responses, terminating) -> recorder(tracked_responses, testee_idx, terminating),
        ),
        item_bank
    )
    true_θ = abilities[:, testee_idx]
    abs_err = sum(abs.(θ .- true_θ))
    @info "convergence" true_θ θ abs_err
end

┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│     0.4651905937702172
│     1.1071205374759037
│   θ =
│    2-element Vector{Float64}:
│     0.5620728626035675
│     0.6629815877828342
└   abs_err = 0.5410212185264197
┌ Info: convergence
│   true_θ =
│    2-element Vector{Float64}:
│     0.0033624072580472823
│     0.8194160266568671
│   θ =
│    2-element Vector{Float64}:
│     -0.009008948337009312
│      0.787947572489276
└   abs_err = 0.043839809762647626


Make a plot showing how the estimated value converges during the CAT.

In [6]:
conv_lines_fig = ability_convergence_lines(recorder; abilities=abilities)
conv_lines_fig

FigureGrid()

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*